In [1]:
from wayne_utils import load_data, save_data
import re
import ast
from tqdm import tqdm

import pandas as pd
tests = load_data( "/home/jiangpeiwen2/jiangpeiwen2/projects/SpCQL_db_process/test.json", "json")       # 2007
trains = load_data( "/home/jiangpeiwen2/jiangpeiwen2/projects/SpCQL_db_process/train.json", "json")     # 7001

test_nl_list = [tests[i]["query"] for i in range(len(tests))]

label_skeleton_list = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/SpCQL/align_db/skeleton_list_label.json", "json")
predict_skeleton_list = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/SpCQL/align_db/skeleton_list_predict.json", "json")
train_fake_list = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/SpCQL/align_db/skeleton_list_train.json", "json")
train_true_list = [trains[i]["cypher"] for i in range(len(trains))]
train_nl_list = [trains[i]["query"] for i in range(len(trains))]

# ablation
non_aligned_intention_skeleton_list = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/SpCQL/ablation/non_aligned_intention_skeleton_list.json", "json")

non_skeleton_intention_train_list = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/SpCQL/intentions/MyMethod_SpCQL_intention_trains_list.json", "json")
non_skeleton_intention_test_list = load_data( "/home/jiangpeiwen2/jiangpeiwen2/NL2GQL/methods/MyMethod/SpCQL/align_db/intentions_aligned.json", "json")

In [5]:
Instruction = """你是一名Neo4j数据库专家，精通Cypher查询语言。下面请你根据用户自然语言查询，结合意图识别结果组件生成的伪查询，生成正真的Cypher查询语句。
自然语言查询部分是原始的用户自然语言表述的查询需求；正式查询部分是能够在Neo4j数据库中执行的语法严谨的Cypher语句。
伪查询：对用户查询进行意图识别后得到的伪Cypher，里面包含了与数据库对齐的标签名和属性名，以及一些基本的Cypher组件。细节如下：
（1）match子句是可能会使用的一些节点、关系、路径对象的列表，其中圆括号中是节点，方括号中是关系，列表中的顺序不代表Cypher语句中的顺序。此外，其中尚未包括最短路径、with子句、多个match并列、Union等复杂情况，如有需要你自行添加。
（2）where子句中是一些可能会使用到的筛选条件。return子句后面是返回格式，已包括对返回格式的大概猜想。
（3）三个部分中的变量名都没有正式命名，你需要结合语句重新将所有节点变量、关系变量、变量替换为英文字符代表的变量名（变量名不是必须的话可以不要），且确保前后一致性。
（4）伪查询仅展示了可能的查询结构，作为你生成正式查询的参考。除非你觉得必要，你不要擅自重写属性名、属性值和标签，因为这是与数据库对齐后的结果。
（5）此外，请你着重考虑关系、路径之间的方向连接、路径长度等要素，重视多查询对象和多返回结果的可能性，以及审查过滤条件和返回格式与自然语言要求的一致性。
"""

Input = """自然语言查询：{NL}
伪查询：{FAKE}
正式查询："""


Instruction_NON_SKELETON = """你是一名Neo4j数据库专家，精通Cypher查询语言。下面请你根据用户自然语言查询，结合意图识别结果，生成Cypher查询语句。
自然语言查询部分是原始的用户自然语言表述的查询需求；正式查询部分是能够在Neo4j数据库中执行的语法严谨的Cypher语句。
意图识别结果：对用户查询进行意图识别后得到的字典数据，里面包含了与数据库对齐的标签名和属性名，以及一些基本的Cypher组件。细节如下：
（1）对象：可能会使用的一些节点、关系、路径对象的属性字典，其中待查询代表用户希望查询的属性值。
（2）约束：包含可能在where子句中使用到的一些约束条件。
（3）返回形式：包含可能在Cypher语句中使用的一些对返回值的要求，例如是否去重、聚合、排序、分页等。
"""

Input_NON_SKELETON = """自然语言查询：{NL}
用户意图：{FAKE}
正式查询："""

In [12]:
def get_generator_prompt( nl_list, fake_lists, true_lists=None ):
    if len(fake_lists) != len(nl_list):
        raise Exception(f"长度不一致{len(fake_lists), len(nl_list)}")
    ret_list = []
    for i in range( len(nl_list) ):
        nl = nl_list[i]
        fake = fake_lists[i]
        if true_lists == None:
            ret_list.append( Instruction + Input.format(NL=nl, FAKE=fake ))
        else:
            ret_list.append( {
                "instruction": Instruction,
                "input": Input.format(NL=nl, FAKE=fake ),
                "output": true_lists[i]
            })
    return ret_list

'''train_ft_list = get_generator_prompt( train_nl_list, train_fake_list, true_lists=train_true_list )
save_data( train_ft_list, "MyMethod_SpCQL_generator_ft.json")
perfect_test_prompts_list = get_generator_prompt( test_nl_list, label_skeleton_list, true_lists=None )

save_data( perfect_test_prompts_list, "MyMethod-Perfect_prompt_list.pickle")
'''

test_prompts_list = get_generator_prompt( test_nl_list, non_aligned_intention_skeleton_list, true_lists=None )
save_data( test_prompts_list, "MyMethod_SpCQL_generator_non_aligned_prompt_list.pickle")


In [10]:
def get_generator_prompt( nl_list, fake_lists, true_lists=None ):
    if len(fake_lists) != len(nl_list):
        raise Exception(f"长度不一致{len(fake_lists), len(nl_list)}")
    ret_list = []
    for i in range( len(nl_list) ):
        nl = nl_list[i]
        fake = fake_lists[i]
        if true_lists == None:
            ret_list.append( Instruction_NON_SKELETON + Input_NON_SKELETON.format(NL=nl, FAKE=fake ))
        else:
            ret_list.append( {
                "instruction": Instruction_NON_SKELETON,
                "input": Input_NON_SKELETON.format(NL=nl, FAKE=fake ),
                "output": true_lists[i]
            })
    return ret_list

'''微调数据集构建
non_aligned_test_prompts_list = get_generator_prompt( train_nl_list, non_skeleton_intention_train_list, true_lists=train_true_list )
save_data( non_aligned_test_prompts_list, "MyMethod_SpCQL_generator_non_aligned_train_list.json")'''

non_aligned_test_prompts_list = get_generator_prompt( test_nl_list, non_skeleton_intention_test_list, true_lists=None )
save_data( non_aligned_test_prompts_list, "MyMethod_SpCQL_generator_non_skeleton_prompt_list.pickle")

In [35]:
from collections import OrderedDict

# cypher子句与关键字字典
cypher_clauses = {
    "MATCH": "用于在图中查找模式。	MATCH (n:Person)-[:FRIEND]->(m)。包含节点、边、路径三类组成的模式，以及用逗号分隔的独立模式", 
    "OPTIONAL MATCH": "类似 MATCH，但允许模式不匹配，返回 NULL。	OPTIONAL MATCH (n)-[:KNOWS]->(m)", 
    "WHERE": "为模式或结果添加过滤条件。	MATCH (n) WHERE n.age > 30 RETURN n", 
    "RETURN": "指定查询结果的输出。	RETURN n.name, n.age", 
    "WITH": "管道式传递中间结果，可用于聚合、过滤、或重命名。	WITH n.name AS name MATCH (m:City)", 
    "UNWIND": "将列表展开为行。	UNWIND [1, 2, 3] AS num RETURN num", 
    "CREATE": "创建节点、关系或子图。	CREATE (n:Person {name: 'Alice'})", 
    "MERGE": "查找或创建节点或关系。	MERGE (n:Person {name: 'Alice'})", 
    "SET": "更新节点或关系的属性或标签。	SET n.age = 30, n:Adult", 
    "DELETE": "删除节点、关系或子图。	MATCH (n) DELETE n", 
    "REMOVE": "移除节点或关系的标签或属性。	MATCH (n) REMOVE n.age", 
    "FOREACH": "针对列表中的每个元素执行一段写操作。	`FOREACH (x IN range(1,10)", 
    "CALL": "调用存储过程或自定义函数。	CALL dbms.procedures()", 
    "LOAD CSV": "导入 CSV 文件中的数据。	LOAD CSV FROM 'file:///data.csv' AS row", 
    "START": "基于图数据库索引查询，主要用于旧版。	START n=node(1) RETURN n", 
    "ORDER BY": "指定查询结果的排序顺序。	RETURN n ORDER BY n.name DESC", 
    "SKIP": "跳过指定数量的行。	RETURN n SKIP 10", 
    "LIMIT": "限制返回的结果行数。	RETURN n LIMIT 5", 
    "UNION": "合并两个查询的结果集（去重）。	MATCH (a:Person) RETURN a.name UNION ...", 
    "UNION ALL": "合并两个查询的结果集（不去重）。	MATCH (a:Person) RETURN a.name UNION ALL", 
}

def split_cypher_clauses(query: str) -> dict:
    clause_pattern = re.compile(
        r"\b(" + r"|".join(re.escape(clause.lower()) for clause in cypher_clauses) + r")\b"
    )
    matches = list(clause_pattern.finditer(query))
    if not matches:
        return {}
    result = OrderedDict({})
    for i, match in enumerate(matches):
        clause = match.group(1)
        start = match.end()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(query)
        result[clause] = query[start:end].strip()

    return result

In [36]:
split_cypher_clauses( "match (:ENTITY{name:'闻香蚀骨'})-[:Relationship{name:'作者'}]->(n) return n.name as name union all match (:ENTITY{name:'闻香蚀骨'})-[:Relationship{name:'类型'}]->(m) return m.name as name union all match (:ENTITY{name:'闻香蚀骨'})-[:Relationship{name:'连载平台'}]->(r) return r.name as name" )

OrderedDict([('match',
              "(:ENTITY{name:'闻香蚀骨'})-[:Relationship{name:'连载平台'}]->(r)"),
             ('return', 'r.name as name'),
             ('union', 'all')])